# Bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Cálculo das prioridades dos critérios 

# Funções

A Função: CalculoPrioridade -> Calcula o autovetor(Prioridade) dos critérios a partir de um argumento do tipo DataFrame (Planilha) que serão lidas do Excel. Derivar as prioridades relativas (relativo pois são medidas em relação aos mesmos critérios) dos critérios. A Prioridade é a pontuação que classifica a importância da alternativa ou cada critério.

In [2]:
def CalculoPrioridade(df):
    soma = df.sum()                                                           #Soma as linhas da matrix de Comparação (DataFrame)
    df_norm = pd.DataFrame()                                                  #Cria um DataFrame vazio, onde será disposta a matrix normalizada
    for i in range(len(df)):                                                  #Ciclo for para percorrer a matrix de comparação e dividir pela soma, para normalizar 
        df_norm[i] = df[df.columns[i]]/soma[i]                                #Normalizaão
    prio = df_norm.mean(axis=1)                                               #Cálcula as médias das colunas gerando o autovetor de Prioridade
    df_norm.columns = df.columns                                              #Insere as colunas da matrix passada(df) para a matrix criada(df_norm) 
    df_norm['Prioridade'] = prio                                              #Insere na marix normalizada uma coluna contendo as prioridades
    df_norm.sort_values(by='Prioridade', inplace = True, ascending=False)     #Rankeia a matrix normalizada pela a ordem de prioridade
    return prio, df_norm

A Função: CalculoConsistencia -> Calcula as variáveis de consistência, CI e CR. O argumento necessário para essa função é o DataFrame Original (não o normalizado) e o auto vetor prioridade (retornardo na função anterior CalculaVetorPrio).

In [3]:
def CalculoConsistencia(vetor, df, RI):
    df_temp = pd.DataFrame()                             #Cria um DataFrame vazio, onde será disposta a matrix temporaria
    for i in range(len(df)):                             #Ciclo for para percorrer a matrix de comparação e multiplicar cada elemento pelo autovetor prioridade
        df_temp[i] = df[df.columns[i]]*vetor[i]
    df_temp.columns = df.columns                        #Insere as colunas da matrix passada(df) para a matrix criada(df_temp)
    total = (df_temp.sum(axis=1)/vetor).sum()           #Soma das prioridade 
    n = len(df)
    Lambda = total/n
    CI = (Lambda-n)/(n-1) # CI = índice de consistência
    CR = CI/RI            # CR = Razão de Consistência
    return df_temp, CI,CR

# Leitura da matriz de comparação

Os critérios de prioridades utilizados serão 4: SDAI, Processo/Projeto, Carga de Energia, Colaborador.
Logo a matrix de comparação será de 4x4

Leitura da matriz de comparação é feita de uma planílha do Excel

In [4]:
matrix_comp = pd.read_excel('Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx', sheet_name='Hierarquizar Risco', index_col=0)
matrix_comp

,SDAI,PROCESSO/PROJETO,CARGA DE ENERGIA,COLABORADOR
Hierarquizar Risco,,,,
SDAI,1.0,5.0000,5.000000,0.333333
PROCESSO/PROJETO,0.2,1.0000,7.002801,0.111111
CARGA DE ENERGIA,0.2,0.1428,1.000000,0.142857
COLABORADOR,3.0,9.0000,7.000000,1.000000


Chamaremos a função CalculoPrioridade para calcular a matrix normalizada e calcular as prioridades dos critérios. 

In [6]:
prioridade, matrix_norm = CalculoPrioridade(matrix_comp)
prioridade

Hierarquizar Risco 
SDAI                0.254357
PROCESSO/PROJETO    0.132896
CARGA DE ENERGIA    0.048719
COLABORADOR         0.564028
dtype: float64

Agora temos um rankeamento desses critérios sabemos que o critério Colaborador é o mais importante (58,40% da importância total), o segundo critério mais importante é O SDAI (26,89% da importância), seguido dos critérios Processo/Projeto com 9,2% e Carga de Energia 5,49%

In [7]:
matrix_norm

,SDAI,PROCESSO/PROJETO,CARGA DE ENERGIA,COLABORADOR,Prioridade
Hierarquizar Risco,,,,,
COLABORADOR,0.681818,0.594342,0.349951,0.63,0.564028
SDAI,0.227273,0.330190,0.249965,0.21,0.254357
PROCESSO/PROJETO,0.045455,0.066038,0.350091,0.07,0.132896
CARGA DE ENERGIA,0.045455,0.009430,0.049993,0.09,0.048719


# Cálculo de consistência

Chamaremos a função CalculoConsistencia() para calcular a consistência da prioridade dos critérios, devolvendo o CI e CR.
Obs: Para teste será retornado o Dataframe também!

Nessa etapa é preciso passar o RI como argumento da Função!

In [8]:
RI = 0.9
matrix_cons, CI, CR = CalculoConsistencia(prioridade, matrix_comp, RI)

In [9]:
matrix_cons

,SDAI,PROCESSO/PROJETO,CARGA DE ENERGIA,COLABORADOR
Hierarquizar Risco,,,,
SDAI,0.254357,0.664479,0.243597,0.188009
PROCESSO/PROJETO,0.050871,0.132896,0.341173,0.062670
CARGA DE ENERGIA,0.050871,0.018978,0.048719,0.080575
COLABORADOR,0.763071,1.196063,0.341036,0.564028


In [10]:
CI

0.24137490225278432

In [11]:
CR

0.26819433583642704

CR < 0,1 Demostrando que o resultado está consistênte!

# Cálculo das prioridades das alternativas 

Função que recebe como argumento o a planílha dos hospitais com o quantitativo das prioridades(Data Frame) e cria uma tabela que compara os argumentos entre eles. No caso as prioridades são: SDAI, Processo/Projeto (PROC), Carga de Energia (ENER), Colaboradores (COLA).  

OBS: Fazer depois, ver se é necessário passar para o Padrão Saaty (1,3,5,7,9).

In [12]:
def TratamentoPrio(data):
    matriz = []
    coluna = []
    name = list(data.columns)
    for i in data[name[0]]:
        for j in data[name[0]]:
            if (j == 0):
                coluna.append(0)
            else:
                coluna.append(i/j)
        matriz.append(coluna)
        coluna = []
    df = pd.DataFrame(matriz, index= data.index, columns=data.index)
    return df 

# Ler os dados das prioridades 

-> SDAI

Lê do excel a tabela com o quantitativo de SDAI de cada hospital. Recebe na variavel "data_sdai" o data frame. 

In [13]:
data_sdai = pd.read_excel('Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx', sheet_name='SDAI', index_col=0)
data_sdai

,SDAI
UNIDADE,
Esperança Recife,4
Norte D'or,15
Copa Star,14
São Rafael,17
Aviccena,1
Memorial,13
Jutta Batista,0
Clínica São Vicente,9
Samer,7


Chama a função TratamentoPrio() criada acima. 

In [15]:
df_sdai = TratamentoPrio(data_sdai)
df_sdai

UNIDADE,Esperança Recife,Norte D'or,Copa Star,São Rafael,Aviccena,Memorial,Jutta Batista,Clínica São Vicente,Samer,São Marcos,UDI,Alphamed,Morumbi,Criança,Itaim,São Lucas
UNIDADE,,,,,,,,,,,,,,,,
Esperança Recife,1.00,0.266667,0.285714,0.235294,4.0,0.307692,0,0.444444,0.571429,0.444444,0.235294,0,4.0,0,0,0.2500
Norte D'or,3.75,1.000000,1.071429,0.882353,15.0,1.153846,0,1.666667,2.142857,1.666667,0.882353,0,15.0,0,0,0.9375
Copa Star,3.50,0.933333,1.000000,0.823529,14.0,1.076923,0,1.555556,2.000000,1.555556,0.823529,0,14.0,0,0,0.8750
São Rafael,4.25,1.133333,1.214286,1.000000,17.0,1.307692,0,1.888889,2.428571,1.888889,1.000000,0,17.0,0,0,1.0625
Aviccena,0.25,0.066667,0.071429,0.058824,1.0,0.076923,0,0.111111,0.142857,0.111111,0.058824,0,1.0,0,0,0.0625
Memorial,3.25,0.866667,0.928571,0.764706,13.0,1.000000,0,1.444444,1.857143,1.444444,0.764706,0,13.0,0,0,0.8125
Jutta Batista,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0,0,0.0000
Clínica São Vicente,2.25,0.600000,0.642857,0.529412,9.0,0.692308,0,1.000000,1.285714,1.000000,0.529412,0,9.0,0,0,0.5625
Samer,1.75,0.466667,0.500000,0.411765,7.0,0.538462,0,0.777778,1.000000,0.777778,0.411765,0,7.0,0,0,0.4375


Entrega a tabela de prioridades para o parâmetro SDAI.

In [16]:
prioridade_sdai, matrix_norm_sdai = CalculoPrioridade(df_sdai)
prioridade_sdai.sort_values(ascending = False)

UNIDADE
  São Rafael             0.138211
  UDI                    0.138211
  São Lucas              0.130081
  Norte D'or             0.121951
  Copa Star              0.113821
  Memorial               0.105691
  Clínica São Vicente    0.073171
  São Marcos             0.073171
  Samer                  0.056911
  Esperança Recife       0.032520
  Aviccena               0.008130
  Morumbi                0.008130
  Jutta Batista          0.000000
  Alphamed               0.000000
  Criança                0.000000
  Itaim                  0.000000
dtype: float64

In [17]:
data_proc = pd.read_excel('Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx', sheet_name='PROC', index_col=0)
data_proc

,Processo / Projeto
UNIDADE,
Esperança Recife,20
Norte D'or,16
Copa Star,12
São Rafael,11
Aviccena,16
Memorial,13
Jutta Batista,11
Clínica São Vicente,11
Samer,14


In [18]:
df_proc = TratamentoPrio(data_proc)
df_proc

UNIDADE,Esperança Recife,Norte D'or,Copa Star,São Rafael,Aviccena,Memorial,Jutta Batista,Clínica São Vicente,Samer,São Marcos,UDI,Alphamed,Morumbi,Criança,Itaim,São Lucas
UNIDADE,,,,,,,,,,,,,,,,
Esperança Recife,1.00,1.2500,1.666667,1.818182,1.2500,1.538462,1.818182,1.818182,1.428571,1.666667,4.0,10.0,20.0,0,6.666667,1.00
Norte D'or,0.80,1.0000,1.333333,1.454545,1.0000,1.230769,1.454545,1.454545,1.142857,1.333333,3.2,8.0,16.0,0,5.333333,0.80
Copa Star,0.60,0.7500,1.000000,1.090909,0.7500,0.923077,1.090909,1.090909,0.857143,1.000000,2.4,6.0,12.0,0,4.000000,0.60
São Rafael,0.55,0.6875,0.916667,1.000000,0.6875,0.846154,1.000000,1.000000,0.785714,0.916667,2.2,5.5,11.0,0,3.666667,0.55
Aviccena,0.80,1.0000,1.333333,1.454545,1.0000,1.230769,1.454545,1.454545,1.142857,1.333333,3.2,8.0,16.0,0,5.333333,0.80
Memorial,0.65,0.8125,1.083333,1.181818,0.8125,1.000000,1.181818,1.181818,0.928571,1.083333,2.6,6.5,13.0,0,4.333333,0.65
Jutta Batista,0.55,0.6875,0.916667,1.000000,0.6875,0.846154,1.000000,1.000000,0.785714,0.916667,2.2,5.5,11.0,0,3.666667,0.55
Clínica São Vicente,0.55,0.6875,0.916667,1.000000,0.6875,0.846154,1.000000,1.000000,0.785714,0.916667,2.2,5.5,11.0,0,3.666667,0.55
Samer,0.70,0.8750,1.166667,1.272727,0.8750,1.076923,1.272727,1.272727,1.000000,1.166667,2.8,7.0,14.0,0,4.666667,0.70


In [19]:
prioridade_proc, matrix_norm_proc = CalculoPrioridade(df_proc)
prioridade_proc.sort_values(ascending = False)

UNIDADE
  Esperança Recife       0.119760
  São Lucas              0.119760
  Norte D'or             0.095808
  Aviccena               0.095808
  Samer                  0.083832
  Memorial               0.077844
  Copa Star              0.071856
  São Marcos             0.071856
  São Rafael             0.065868
  Jutta Batista          0.065868
  Clínica São Vicente    0.065868
  UDI                    0.029940
  Itaim                  0.017964
  Alphamed               0.011976
  Morumbi                0.005988
  Criança                0.000000
dtype: float64

In [20]:
data_ener = pd.read_excel('Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx', sheet_name='ENER', index_col=0)
data_ener

,Carga de Energia
UNIDADE,
Esperança Recife,12
Norte D'or,10
Copa Star,12
São Rafael,15
Aviccena,15
Memorial,7
Jutta Batista,13
Clínica São Vicente,8
Samer,13


In [21]:
df_ener = TratamentoPrio(data_ener)
df_ener

UNIDADE,Esperança Recife,Norte D'or,Copa Star,São Rafael,Aviccena,Memorial,Jutta Batista,Clínica São Vicente,Samer,São Marcos,UDI,Alphamed,Morumbi,Criança,Itaim,São Lucas
UNIDADE,,,,,,,,,,,,,,,,
Esperança Recife,1.000000,1.2,1.000000,0.800000,0.800000,1.714286,0.923077,1.500,0.923077,2.4,6.0,0,0,0,12.0,1.000000
Norte D'or,0.833333,1.0,0.833333,0.666667,0.666667,1.428571,0.769231,1.250,0.769231,2.0,5.0,0,0,0,10.0,0.833333
Copa Star,1.000000,1.2,1.000000,0.800000,0.800000,1.714286,0.923077,1.500,0.923077,2.4,6.0,0,0,0,12.0,1.000000
São Rafael,1.250000,1.5,1.250000,1.000000,1.000000,2.142857,1.153846,1.875,1.153846,3.0,7.5,0,0,0,15.0,1.250000
Aviccena,1.250000,1.5,1.250000,1.000000,1.000000,2.142857,1.153846,1.875,1.153846,3.0,7.5,0,0,0,15.0,1.250000
Memorial,0.583333,0.7,0.583333,0.466667,0.466667,1.000000,0.538462,0.875,0.538462,1.4,3.5,0,0,0,7.0,0.583333
Jutta Batista,1.083333,1.3,1.083333,0.866667,0.866667,1.857143,1.000000,1.625,1.000000,2.6,6.5,0,0,0,13.0,1.083333
Clínica São Vicente,0.666667,0.8,0.666667,0.533333,0.533333,1.142857,0.615385,1.000,0.615385,1.6,4.0,0,0,0,8.0,0.666667
Samer,1.083333,1.3,1.083333,0.866667,0.866667,1.857143,1.000000,1.625,1.000000,2.6,6.5,0,0,0,13.0,1.083333


In [22]:
prioridade_ener, matrix_norm_ener = CalculoPrioridade(df_ener)
prioridade_ener.sort_values(ascending = False)

UNIDADE
  São Rafael             0.120
  Aviccena               0.120
  Jutta Batista          0.104
  Samer                  0.104
  Esperança Recife       0.096
  Copa Star              0.096
  São Lucas              0.096
  Norte D'or             0.080
  Clínica São Vicente    0.064
  Memorial               0.056
  São Marcos             0.040
  UDI                    0.016
  Itaim                  0.008
  Alphamed               0.000
  Morumbi                0.000
  Criança                0.000
dtype: float64

In [23]:
data_cola = pd.read_excel('Planilha de Ranqueamento de Riscos de Unidades_teste.xlsx', sheet_name='COLA', index_col=0)
data_cola

,Colaboradores
UNIDADE,
São Lucas,8
São Rafael,4
UDI,4
Norte D'or,4
São Marcos,5
Memorial,4
Copa Star,3
Samer,6
Clínica São Vicente,2


In [24]:
df_cola = TratamentoPrio(data_cola)
df_cola

UNIDADE,São Lucas,São Rafael,UDI,Norte D'or,São Marcos,Memorial,Copa Star,Samer,Clínica São Vicente,Esperança Recife,Aviccena,Morumbi,Alphamed,Criança,Jutta Batista,Itaim
UNIDADE,,,,,,,,,,,,,,,,
São Lucas,1.000,2.00,2.00,2.00,1.6,2.00,2.666667,1.333333,4.0,1.6,2.00,8.0,8.0,8.0,0,0
São Rafael,0.500,1.00,1.00,1.00,0.8,1.00,1.333333,0.666667,2.0,0.8,1.00,4.0,4.0,4.0,0,0
UDI,0.500,1.00,1.00,1.00,0.8,1.00,1.333333,0.666667,2.0,0.8,1.00,4.0,4.0,4.0,0,0
Norte D'or,0.500,1.00,1.00,1.00,0.8,1.00,1.333333,0.666667,2.0,0.8,1.00,4.0,4.0,4.0,0,0
São Marcos,0.625,1.25,1.25,1.25,1.0,1.25,1.666667,0.833333,2.5,1.0,1.25,5.0,5.0,5.0,0,0
Memorial,0.500,1.00,1.00,1.00,0.8,1.00,1.333333,0.666667,2.0,0.8,1.00,4.0,4.0,4.0,0,0
Copa Star,0.375,0.75,0.75,0.75,0.6,0.75,1.000000,0.500000,1.5,0.6,0.75,3.0,3.0,3.0,0,0
Samer,0.750,1.50,1.50,1.50,1.2,1.50,2.000000,1.000000,3.0,1.2,1.50,6.0,6.0,6.0,0,0
Clínica São Vicente,0.250,0.50,0.50,0.50,0.4,0.50,0.666667,0.333333,1.0,0.4,0.50,2.0,2.0,2.0,0,0


In [25]:
prioridade_cola, matrix_norm_cola = CalculoPrioridade(df_cola)
prioridade_cola.sort_values(ascending = False)

UNIDADE
  São Lucas              0.153846
  Samer                  0.115385
  São Marcos             0.096154
  Esperança Recife       0.096154
  São Rafael             0.076923
  UDI                    0.076923
  Norte D'or             0.076923
  Memorial               0.076923
  Aviccena               0.076923
  Copa Star              0.057692
  Clínica São Vicente    0.038462
  Morumbi                0.019231
  Alphamed               0.019231
  Criança                0.019231
  Jutta Batista          0.000000
  Itaim                  0.000000
dtype: float64

Decisão Final

In [26]:
matrix_decisao = pd.DataFrame([prioridade_sdai, prioridade_proc, prioridade_ener, prioridade_cola], 
                              index=['SDAI', 'Processo', 'Energia', 'Colaboradores'])
matrix_decisao = matrix_decisao.transpose()
matrix_decisao

,SDAI,Processo,Energia,Colaboradores
UNIDADE,,,,
Esperança Recife,0.032520,0.119760,0.096,0.096154
Norte D'or,0.121951,0.095808,0.080,0.076923
Copa Star,0.113821,0.071856,0.096,0.057692
São Rafael,0.138211,0.065868,0.120,0.076923
Aviccena,0.008130,0.095808,0.120,0.076923
Memorial,0.105691,0.077844,0.056,0.076923
Jutta Batista,0.000000,0.065868,0.104,0.000000
Clínica São Vicente,0.073171,0.065868,0.064,0.038462
Samer,0.056911,0.083832,0.104,0.115385


In [27]:
matrix_decisao['Colaboradores'].sort_values(ascending = False)

UNIDADE
  São Lucas              0.153846
  Samer                  0.115385
  Esperança Recife       0.096154
  São Marcos             0.096154
  Norte D'or             0.076923
  São Rafael             0.076923
  Aviccena               0.076923
  Memorial               0.076923
  UDI                    0.076923
  Copa Star              0.057692
  Clínica São Vicente    0.038462
  Alphamed               0.019231
  Morumbi                0.019231
  Criança                0.019231
  Jutta Batista          0.000000
  Itaim                  0.000000
Name: Colaboradores, dtype: float64

In [28]:
peso_criterio = prioridade
peso_criterio

Hierarquizar Risco 
SDAI                0.254357
PROCESSO/PROJETO    0.132896
CARGA DE ENERGIA    0.048719
COLABORADOR         0.564028
dtype: float64

In [29]:
(matrix_decisao['SDAI']*peso_criterio['SDAI']).sort_values(ascending=False)

UNIDADE
  São Rafael             0.035155
  UDI                    0.035155
  São Lucas              0.033087
  Norte D'or             0.031019
  Copa Star              0.028951
  Memorial               0.026883
  Clínica São Vicente    0.018611
  São Marcos             0.018611
  Samer                  0.014476
  Esperança Recife       0.008272
  Aviccena               0.002068
  Morumbi                0.002068
  Jutta Batista          0.000000
  Alphamed               0.000000
  Criança                0.000000
  Itaim                  0.000000
Name: SDAI, dtype: float64

In [30]:
matrix_decisao['Processo']*peso_criterio['PROCESSO/PROJETO']

UNIDADE
  Esperança Recife       0.015916
  Norte D'or             0.012733
  Copa Star              0.009549
  São Rafael             0.008754
  Aviccena               0.012733
  Memorial               0.010345
  Jutta Batista          0.008754
  Clínica São Vicente    0.008754
  Samer                  0.011141
  São Marcos             0.009549
  UDI                    0.003979
  Alphamed               0.001592
  Morumbi                0.000796
  Criança                0.000000
  Itaim                  0.002387
  São Lucas              0.015916
Name: Processo, dtype: float64

In [31]:
matrix_decisao['Energia']*peso_criterio['CARGA DE ENERGIA']

UNIDADE
  Esperança Recife       0.004677
  Norte D'or             0.003898
  Copa Star              0.004677
  São Rafael             0.005846
  Aviccena               0.005846
  Memorial               0.002728
  Jutta Batista          0.005067
  Clínica São Vicente    0.003118
  Samer                  0.005067
  São Marcos             0.001949
  UDI                    0.000780
  Alphamed               0.000000
  Morumbi                0.000000
  Criança                0.000000
  Itaim                  0.000390
  São Lucas              0.004677
Name: Energia, dtype: float64

In [32]:
matrix_decisao['Colaboradores']*peso_criterio['COLABORADOR']

UNIDADE
  Esperança Recife       0.054233
  Norte D'or             0.043387
  Copa Star              0.032540
  São Rafael             0.043387
  Aviccena               0.043387
  Memorial               0.043387
  Jutta Batista          0.000000
  Clínica São Vicente    0.021693
  Samer                  0.065080
  São Marcos             0.054233
  UDI                    0.043387
  Alphamed               0.010847
  Morumbi                0.010847
  Criança                0.010847
  Itaim                  0.000000
  São Lucas              0.086774
Name: Colaboradores, dtype: float64

In [33]:
matrix_decisao_final = pd.DataFrame()

In [34]:
matrix_decisao_final['SDAI'] = matrix_decisao['SDAI']*peso_criterio['SDAI']
matrix_decisao_final['Processo'] = matrix_decisao['Processo']*peso_criterio['PROCESSO/PROJETO']
matrix_decisao_final['Energia'] = matrix_decisao['Energia']*peso_criterio['CARGA DE ENERGIA']
matrix_decisao_final['Colaboradores'] = matrix_decisao['Colaboradores']*peso_criterio['COLABORADOR']

In [35]:
matrix_decisao_final['Soma'] = matrix_decisao_final['SDAI'] + matrix_decisao_final['Processo'] + matrix_decisao_final['Energia']
+ matrix_decisao_final['Colaboradores']

UNIDADE
  Esperança Recife       0.054233
  Norte D'or             0.043387
  Copa Star              0.032540
  São Rafael             0.043387
  Aviccena               0.043387
  Memorial               0.043387
  Jutta Batista          0.000000
  Clínica São Vicente    0.021693
  Samer                  0.065080
  São Marcos             0.054233
  UDI                    0.043387
  Alphamed               0.010847
  Morumbi                0.010847
  Criança                0.010847
  Itaim                  0.000000
  São Lucas              0.086774
Name: Colaboradores, dtype: float64

In [36]:
matrix_decisao_final['Soma'].sort_values(ascending = False)

UNIDADE
  São Lucas              0.053680
  São Rafael             0.049755
  Norte D'or             0.047649
  Copa Star              0.043178
  Memorial               0.039957
  UDI                    0.039913
  Samer                  0.030683
  Clínica São Vicente    0.030483
  São Marcos             0.030110
  Esperança Recife       0.028865
  Aviccena               0.020647
  Jutta Batista          0.013820
  Morumbi                0.002864
  Itaim                  0.002777
  Alphamed               0.001592
  Criança                0.000000
Name: Soma, dtype: float64